# Error propagation

## Tasks

Solve the tasks below.
For each task, give reasons for your solution by commenting in the notebook.
In conclusion, summarize your findings and contextualize them. What have you learned? Do the results make sense?

Your results will be examined for plagiarism. Please use your own plot styles, articulate your own thoughts, and present your own experimental approaches.

a) Propagate uncertainties for the following expressions using [SymPy](https://www.sympy.org) following the examples for [uncorrelated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation.ipynb?flush_cache=false) and [correlated variables](https://nbviewer.jupyter.org/urls/www.physi.uni-heidelberg.de/Einrichtungen/FP/Datenanalyse/FP_Gaussian_error_propagation_corr.ipynb?flush_cache=false) from the FP web page.

i) Find expressions for the absolute uncertainty $\sigma_z$ for $z = x + y$ and $z = x - y$ 

ii) Find expressions for the relative uncertainty $\sigma_z / z$ for $z = x \cdot y, \; z = x / y$ and $z = x^n y^n$

iii) The acceleration of gravity with a simple pendulum is given by the following formula:
$$g = 4  \pi^2 \frac{L}{T^2}$$
The relevant variables are the length $L$ of the pendulum and the period $T$ with the corresponding errors $\sigma_L$ and $\sigma_T$.

iv) The energy of a rotating object is given by:
$$E = \frac{1}{2} I \omega^2$$
The relevant variables are the angular velocity $\omega$ and the moment of interia $I$ with the corresponding errors $\sigma_\omega$ and $\sigma_I$.

b) The radius $r$ and the height $h$ of a cylinder have been measured to $r = 5$ cm and $h = 2$ cm. The uncertainty for both measurements is $\sigma = 0.1$ cm. Determine the volume of the cylinder and its uncertainty assuming (i) that both measurements are uncorrelated and (ii) that both measurements are fully correlated.

c) The scattering angle and the radial distance of a certain particle can be determined from a position measurement $(x,y)$ 
$$r = \sqrt{x^2 + y^2}, \quad \theta = \mathrm{atan2}(y, x)$$
You find more on the [atan2](https://en.wikipedia.org/wiki/Atan2) function on wikipedia. The position ($x$,$y$) is measured with the corresponding uncertainties $\sigma_x$ and $\sigma_y$. Write a python function that returns the covariance matrix $U$ of $r$ and $\theta$ for a given covariance matrix $V$ of $x$ and $y$. Determine $U$ under the assumption that $x$ and $y$ are uncorrelated. Hint: The formulas you need can be found in the script.


### Solution

In [2]:
from sympy import *
from IPython.display import display, Latex

#### a) Gaussian Error Propagation of uncertainties for 2 uncorrelated variables

In [3]:
def error_prop(f, vars):
    """
    Symbolic Gaussian error propagation.
    
    Arguments:
    f: formula (sympy expression)
    vars: list of independent variables and corresponding uncertainties [(x1, sigma_x1), (x2, sigma_x2), ...]
    
    return value: sympy expression for the uncertainty of the calculated expression f
    
    """
    sum = S(0) # empty SymPy expression
    for (x, sigma) in vars:
        sum += diff(f, x)**2 * sigma**2       #Addition of partial deviations * each standard deviation
    return sqrt(simplify(sum))

i) $z=x+y$, $\sigma_z=\sqrt{(\sigma_x)^2+(\sigma_y)^2}$

In [6]:
x, y, sigma_x, sigma_y=symbols('x, y, sigma_x, sigma_y', positive=True)  # Define sympy symbols and formula for the z
z=x+y

sigma_z = error_prop(z, [(x, sigma_x), (y, sigma_y)])                    # Determine expression for the uncertainty \sigma_z from Gaussian error propagation
result = "$${} = {}$$".format("\\sigma_z", latex(sigma_z))
display(Latex(result))

<IPython.core.display.Latex object>

ii) find relative uncertainty $\sigma_z/z$ for $z=xy$, $z=x/y$, $z=x^ny^n$

In [9]:
x, y, sigma_x, sigma_y=symbols('x, y, sigma_x, sigma_y', positive=True) 
z=x*y

sigma_z = error_prop(z, [(x, sigma_x), (y, sigma_y)]) 
result = "$${} = {}$$".format("\\sigma_z/z", latex(sigma_z/z))
display(Latex(result))

<IPython.core.display.Latex object>

In [10]:
x, y, sigma_x, sigma_y=symbols('x, y, sigma_x, sigma_y', positive=True) 
z=x/y

sigma_z = error_prop(z, [(x, sigma_x), (y, sigma_y)]) 
result = "$${} = {}$$".format("\\sigma_z/z", latex(sigma_z/z))
display(Latex(result))

<IPython.core.display.Latex object>

In [11]:
n, x, y, sigma_x, sigma_y=symbols('n, x, y, sigma_x, sigma_y', positive=True)
z=x**n*y**n

sigma_z = error_prop(z, [(x, sigma_x), (y, sigma_y)]) 
result = "$${} = {}$$".format("\\sigma_z/z", latex(sigma_z/z))
display(Latex(result))

<IPython.core.display.Latex object>

iii) $g = 4  \pi^2 \frac{L}{T^2}$ with $\sigma_L$, $\sigma_T$

In [19]:
import numpy as np
pi, L, T, sigma_L, sigma_T=symbols('pi, L, T, sigma_L, sigma_T', positive=True)
g=4*pi**2*L/T**2

sigma_g = error_prop(g, [(L, sigma_L), (T, sigma_T)]) 
result = "$${} = {}$$".format("\\sigma_g", latex(sigma_g))
display(Latex(result))

<IPython.core.display.Latex object>

iv) $E = \frac{1}{2} I \omega^2$ with $\sigma_I$, $\sigma_\omega$

In [18]:
I, w, sigma_I, sigma_w=symbols('I, w, sigma_I, sigma_w', positive=True)
E0=I*w**2

sigma_E0 = error_prop(E0, [(I, sigma_I), (w, sigma_w)]) 
result = "$${} = {}$$".format("\\sigma_E", latex(sigma_E0/2))
display(Latex(result))

<IPython.core.display.Latex object>

#### Gaussian Error Propagation of uncertainties for 2 correlated variables

In [4]:
def error_prop_corr(f, x, V):
    """
    f: function f = f(x[0], x[1], ...)
    x: list of variables
    V: covariance matrix (python 2d list)
    """
    sum = S(0) # empty sympy expression
    for i in range(len(x)):
        for j in range(len(x)):
            sum += diff(f, x[i]) * diff(f, x[j]) * V[i][j] 
    return sqrt(simplify(sum))

i) $z=x+y$

In [33]:
x, y, sigma_x, sigma_y = symbols('x, y, sigma_x, sigma_y', positive=True)
rho = Symbol("rho", real=True)

# covariance matrix
C = [[sigma_x**2, rho * sigma_x * sigma_y], [rho * sigma_x * sigma_y, sigma_y**2]]

z = x + y
vars = [x, y]    #list of variables
sigma_z = error_prop_corr(z, vars, C)
sigma_z

sqrt(2*rho*sigma_x*sigma_y + sigma_x**2 + sigma_y**2)

ii) $\sigma_z/z$ for $z=xy$, $z=x/y$, $z=x^ny^n$

In [35]:
x, y, sigma_x, sigma_y = symbols('x, y, sigma_x, sigma_y', positive=True)
z1 = x*y
z2=x/y
vars = [x, y]
sigma_z1 = error_prop_corr(z1, vars, C) / z1
sigma_z1

sqrt(2*rho*sigma_x*sigma_y*x*y + sigma_x**2*y**2 + sigma_y**2*x**2)/(x*y)

In [36]:
sigma_z2 = error_prop_corr(z2, vars, C) / z2
sigma_z2

sqrt(-2*rho*sigma_x*sigma_y*x*y + sigma_x**2*y**2 + sigma_y**2*x**2)/(x*y)

In [37]:
n, x, y, sigma_x, sigma_y = symbols('n, x, y, sigma_x, sigma_y', positive=True)
z3=x**n*y**n
vars = [x, y]
sigma_z3 = error_prop_corr(z3, vars, C) / z3
sigma_z3

n*sqrt((x*y)**(2*n - 2)*(2*rho*sigma_x*sigma_y*x*y + sigma_x**2*y**2 + sigma_y**2*x**2))/(x**n*y**n)

iii) $g = 4  \pi^2 \frac{L}{T^2}$ with $\sigma_L$, $\sigma_T$

In [38]:
pi, L, T, sigma_L, sigma_T = symbols('pi, L, T, sigma_L, sigma_T', positive=True)
rho = Symbol("rho", real=True)
C = [[sigma_L**2, rho * sigma_L * sigma_T], [rho * sigma_L * sigma_T, sigma_T**2]]

g = 4*pi**2*L/T**2
vars = [L, T] 
sigma_g = error_prop_corr(g, vars, C)
sigma_g

4*pi**2*sqrt(4*L**2*sigma_T**2 - 4*L*T*rho*sigma_L*sigma_T + T**2*sigma_L**2)/T**3

iv) $E = \frac{1}{2} I \omega^2$ with $\sigma_I$, $\sigma_\omega$

In [39]:
I, w, sigma_I, sigma_w = symbols('I, w, sigma_I, sigma_w', positive=True)
rho = Symbol("rho", real=True)
C = [[sigma_I**2, rho * sigma_I * sigma_w], [rho * sigma_I * sigma_w, sigma_w**2]]
E=I*w**2/2
vars = [I, w]
sigma_E = error_prop_corr(E, vars, C)
sigma_E

w*sqrt(I**2*sigma_w**2 + I*rho*sigma_I*sigma_w*w + sigma_I**2*w**2/4)

b) The radius $r$ and the height $h$ of a cylinder have been measured to $r = 5$ cm and $h = 2$ cm. The uncertainty for both measurements is $\sigma = 0.1$ cm. Determine the volume of the cylinder and its uncertainty assuming (i) that both measurements are uncorrelated and (ii) that both measurements are fully correlated.


In [40]:
r_meas = 5   # in cm 
sigma_r_meas = 0.1
h_meas = 2
sigma_h_meas = 0.1

In [57]:
# both measurements are uncorrelated
r, h, sigma_r, sigma_h = symbols('r, h, sigma_r, sigma_h', positive=True)
V = pi * r**2 * h
sigma_V = error_prop(V, [(r, sigma_r), (h, sigma_h)])
fehler = "$${} = {}$$".format("\\sigma_V", latex(sigma_V))
volume = f"$$V = {latex(V)}$$"
display(Latex(volume))
display(Latex(fehler))

V = np.pi * r**2 * h                                          # V, sigma_V überschreiben damit symbolische Ausdrücke in Gleitkommazahlen gesetzt werden
sigma_V = error_prop(V, [(r, sigma_r), (h, sigma_h)])
                                                              # Die Methode .evalf() konvertiert einen symbolischen Ausdruck in einen numerischen Wert (Floating Point)
central_value = V.subs([(r,r_meas), (h, h_meas)]).evalf()     # .sub benutzen um symbolische Variablen durch konkrete Werte zu ersetzen
sigma = sigma_V.subs([(r, r_meas), (sigma_r, sigma_r_meas), (h, h_meas), (sigma_h, sigma_h_meas)]).evalf()
result = f"$$V= ({central_value:.1f} \\pm {sigma:.1f}) \\, \\mathrm{{cm}}^3$$"    # :1f 1. Dezimalstelle
display(Latex(result))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [55]:
# both measurements are correlated
r, h, sigma_r, sigma_h = symbols('r, h, sigma_r, sigma_h', positive=True)
V = pi * r**2 * h
volume = f"$$V = {latex(V)}$$"
display(Latex(volume))
rho = Symbol("rho", real=True)
C = [[sigma_r**2, rho * sigma_r * sigma_h], [rho * sigma_r * sigma_h, sigma_h**2]]
vars = [r, h]
sigma_V = error_prop_corr(V, vars, C)
fehler = "$${} = {}$$".format("\\sigma_V", latex(sigma_V))
display(Latex(fehler))

V = np.pi * r**2 * h                                          
sigma_V = error_prop_corr(V, vars, C)
rho0=1         # fully correlated \rho=1
central_value = V.subs([(r,r_meas), (h, h_meas)]).evalf()     
sigma = sigma_V.subs([(r, r_meas), (sigma_r, sigma_r_meas), (h, h_meas), (sigma_h, sigma_h_meas), (rho, rho0)]).evalf()   #ersetze rho in rho0
result = f"$$V = ({central_value:.1f} \\pm {sigma:.1f}) \\, \\mathrm{{cm}}^3$$"    
display(Latex(result))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

c) $r = \sqrt{x^2 + y^2}, \quad \theta = \mathrm{atan2}(y, x)=\mathrm{arctan}(y/x)$. The position ($x$,$y$) is measured with the corresponding uncertainties $\sigma_x$ and $\sigma_y$. Write a python function that returns the covariance matrix $U$ of $r$ and $\theta$ for a given covariance matrix $V$ of $x$ and $y$. Determine $U$ under the assumption that $x$ and $y$ are uncorrelated. Hint: The formulas you need can be found in the script.

For $\vec{y}=(r, \theta)=:(f_1, f_2)$, $\vec{x}=(x,y)=:(x_1,x_2)$ as in script, $G_{ki}:=\frac{\delta f_k}{\delta x_i}$ Jacobi matrix, C the covariance matrix of $\vec{x}$
$$\mathbf{C}=\left(\begin{array}{cc}  
	\sigma_x^2 & \text{cov}(x,y)\\
	\text{cov}(x,y) & \sigma_y^2
	\end{array}\right) = \left(\begin{array}{cc}  
	\sigma_x^2 & \rho \sigma_x\sigma_y\\
	\rho \sigma_x\sigma_y & \sigma_y^2
	\end{array}\right) \qquad \mathbf{G}=\left(\begin{array}{cc}  
	x/r & y/r\\
	-y/r^2 & x/r^2
	\end{array}\right)$$
The covariance matrix of U of $r$ and $\theta$ is given by: $U=G \cdot C \cdot G^T$

In [78]:
#r=np.sqrt(x**2+y**2)
#o=np.arctan(y/x)
def covariance_matrix_of_U(x, y, sigma_x, sigma_y, rho):
    r=np.sqrt(x**2+y**2)
    C = np.array([[sigma_x**2, rho*sigma_x*sigma_y], 
                  [rho*sigma_x*sigma_y, sigma_y**2]])         # covariance matrix J
    G=np.array([[x/r, y/r], 
                [-y/r**2, x/r**2]])          # Jacobi-Matrix
    U = G @ C @ G.T      # covariance matrix U

    return U

Test

In [83]:
x = 3.0 
y = 4.0
sigma_x=0.1
sigma_y=0.1
rho=1

U = covariance_matrix_of_U(x, y,sigma_x, sigma_y, rho)
print(U)


[[ 1.96e-02 -5.60e-04]
 [-5.60e-04  1.60e-05]]


if x, y are uncorrelated: $\rho=0$:

In [82]:
x = 3.0 
y = 4.0
sigma_x=0.1
sigma_y=0.1
rho=0
              
U = covariance_matrix_of_U(x, y,sigma_x, sigma_y, rho)
print(U)

[[1.00000000e-02 5.98826544e-20]
 [9.92261828e-21 4.00000000e-04]]
